# Model Application

In this part of te project we import new data and the models we have created to predict labels. With will upload the results to AWS S3 for be displayed in our frontend

In [1]:
import pandas as pd
import pickle

In [2]:
# First we must load Column Transformer

ct_filename = 'Model/ct.pkl'
ct = pickle.load(open(ct_filename, 'rb'))

In [14]:
# Read csv with previuos data, we will need to create some labels based on variations respect previous values

diagnosis_filename = 'Data/diagnosis_model.csv'
diagnosis_prev = pd.read_csv(diagnosis_filename, index_col=0)
diagnosis_prev = diagnosis_prev.drop(columns=['Nombre', 'diagnostico'])
diagnosis_prev.head()

,machine,point,time,max_mms,rms_mms,max_acc,med_acc,rms_acc,kur_acc,cf_acc,...,e_range,f_range,g_range,h_range,i_range,j_range,max_dem,rms_dem,Id_Estado_Activo,Id_Estado_Activo_fixed
1,S3-M38,M-LOA-H,2019-06-18 12:38:26.205,9.2617,4.066749,0.46940,0.347729,0.148627,0.388228,3.158231,...,0.0,0.0,0.0,0.0,0.0,0.0,0.31838,0.130597,2.0,2.0
3,S3-M38,M-LOA-V,2019-06-18 12:38:41.249,6.4828,2.278911,0.43131,0.349279,0.147235,0.579381,2.929393,...,0.0,0.0,0.0,0.0,0.0,0.0,0.22062,0.103748,2.0,2.0
4,S3-M38,M-LA-H,2019-06-18 12:38:57.866,7.4407,2.977928,0.64757,0.334221,0.141516,0.324311,4.575943,...,0.0,0.0,0.0,0.0,0.0,0.0,0.53557,0.168171,2.0,2.0
7,S3-M38,M-LA-V,2019-06-18 12:39:08.660,6.9886,2.452952,0.72522,0.339139,0.146303,0.600235,4.956958,...,0.0,0.0,0.0,0.0,0.0,0.0,0.55888,0.169940,2.0,2.0
8,S3-M38,M-LA-A,2019-06-18 12:39:20.397,7.3701,3.351391,0.48182,0.343762,0.145565,0.395430,3.309999,...,0.0,0.0,0.0,0.0,0.0,0.0,0.24158,0.095285,2.0,2.0


In [11]:
# Read csv with new values to predict labels

diagnosis_pred_filename = 'Data/diagnosis_2021.csv'
diagnosis_pred = pd.read_csv(diagnosis_pred_filename, index_col =0)
diagnosis_pred.head(10)

,machine,point,time,max_mms,rms_mms,max_acc,med_acc,rms_acc,kur_acc,cf_acc,...,c_range,d_range,e_range,f_range,g_range,h_range,i_range,j_range,max_dem,rms_dem
10320,S3-M2,M-LOA-H,2021-04-06 10:42:01.084,3.9192,1.329374,0.41243,0.260998,0.086605,0.368448,4.762184,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.36541,0.146129
10321,S3-M2,M-LOA-V,2021-04-06 10:42:11.393,6.8647,3.543557,0.38845,0.276361,0.095755,0.026829,4.056714,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.36867,0.159407
10322,S3-M2,M-LA-H,2021-04-06 10:42:22.163,3.1336,1.231007,0.34547,0.263488,0.086960,0.027979,3.972731,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.44539,0.190762
10323,S3-M2,M-LA-V,2021-04-06 10:42:30.994,6.9454,3.938512,0.40552,0.284797,0.101715,0.002531,3.986830,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.39386,0.209035
10324,S3-M2,M-LA-A,2021-04-06 10:42:41.913,4.0761,1.442913,0.25459,0.230750,0.067403,0.128018,3.777112,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.24223,0.107505
10325,S3-M2,V-LA-H,2021-04-06 10:42:52.612,3.4483,1.117077,0.52575,0.344202,0.147478,0.150945,3.564950,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.50852,0.247369
10326,S3-M2,V-LA-V,2021-04-06 10:43:03.192,3.9093,1.699599,0.55850,0.344702,0.149701,0.082864,3.730771,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.60670,0.278389
10327,S3-M2,V-LOA-H,2021-04-06 10:43:15.581,1.6046,0.518784,0.61104,0.352737,0.156591,0.066179,3.902145,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.64367,0.292268
10328,S3-M2,V-LOA-V,2021-04-06 10:43:25.313,11.8500,3.080179,0.72477,0.346719,0.151017,0.050923,4.799271,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.64791,0.275261
10329,S3-M2,V-LOA-A,2021-04-06 10:43:35.593,1.5914,0.581681,0.36238,0.281129,0.098507,0.128110,3.678735,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.37572,0.181459


In [40]:
# We must add two columns with dummy data before concatenate with
diagnosis_pred['Id_Estado_Activo'] = 1.0
diagnosis_pred['Id_Estado_Activo_fixed'] = 1.0
diagnosis_pred.head()

,machine,point,time,max_mms,rms_mms,max_acc,med_acc,rms_acc,kur_acc,cf_acc,...,e_range,f_range,g_range,h_range,i_range,j_range,max_dem,rms_dem,Id_Estado_Activo,Id_Estado_Activo_fixed
10320,S3-M2,M-LOA-H,2021-04-06 10:42:01.084,3.9192,1.329374,0.41243,0.260998,0.086605,0.368448,4.762184,...,0.0,0.0,0.0,0.0,0.0,0.0,0.36541,0.146129,1.0,1.0
10321,S3-M2,M-LOA-V,2021-04-06 10:42:11.393,6.8647,3.543557,0.38845,0.276361,0.095755,0.026829,4.056714,...,0.0,0.0,0.0,0.0,0.0,0.0,0.36867,0.159407,1.0,1.0
10322,S3-M2,M-LA-H,2021-04-06 10:42:22.163,3.1336,1.231007,0.34547,0.263488,0.086960,0.027979,3.972731,...,0.0,0.0,0.0,0.0,0.0,0.0,0.44539,0.190762,1.0,1.0
10323,S3-M2,M-LA-V,2021-04-06 10:42:30.994,6.9454,3.938512,0.40552,0.284797,0.101715,0.002531,3.986830,...,0.0,0.0,0.0,0.0,0.0,0.0,0.39386,0.209035,1.0,1.0
10324,S3-M2,M-LA-A,2021-04-06 10:42:41.913,4.0761,1.442913,0.25459,0.230750,0.067403,0.128018,3.777112,...,0.0,0.0,0.0,0.0,0.0,0.0,0.24223,0.107505,1.0,1.0


In [54]:
result = diagnosis_prev.append(diagnosis_pred, sort=False)
result.tail()

,machine,point,time,max_mms,rms_mms,max_acc,med_acc,rms_acc,kur_acc,cf_acc,...,e_range,f_range,g_range,h_range,i_range,j_range,max_dem,rms_dem,Id_Estado_Activo,Id_Estado_Activo_fixed
12895,S5-60-M1,V-LA-H,2021-04-07 12:15:06.604,4.0158,1.300346,2.1108,0.662715,0.551288,0.005076,3.828850,...,0.0000,0.32248,0.14730,0.0000,0.00000,0.0,2.3101,1.022200,1.0,1.0
12896,S5-60-M1,V-LA-V,2021-04-07 12:15:16.954,7.5295,2.813692,2.0631,0.650348,0.530640,0.014530,3.887949,...,0.0000,0.12103,0.00000,0.1107,0.10888,0.0,2.5691,1.069902,1.0,1.0
12897,S5-60-M1,V-LOA-H,2021-04-07 12:15:28.784,8.0457,1.741759,1.8851,0.620282,0.481130,0.055645,3.918069,...,0.2014,0.28060,0.19783,0.0000,0.00000,0.0,2.0866,0.902661,1.0,1.0
12898,S5-60-M1,V-LOA-V,2021-04-07 12:15:39.724,4.0366,1.287759,1.5529,0.597956,0.445751,0.157203,3.483784,...,0.0000,0.00000,0.00000,0.0000,0.00000,0.0,1.8123,0.887511,1.0,1.0
12899,S5-60-M1,V-LOA-A,2021-04-07 12:15:51.904,4.2682,1.502579,1.0482,0.466053,0.270979,0.083215,3.868201,...,0.0000,0.00000,0.00000,0.0000,0.00000,0.0,1.1964,0.548979,1.0,1.0


In [55]:

result = result.sort_values(by='time')
result['prev_diag'] = result.groupby(['machine','point'])['Id_Estado_Activo'].shift(fill_value=1)
result['pct_change_acc'] = result.groupby(['machine','point']).max_acc.pct_change(fill_method='ffill' )
result['mean_mms_prev'] = result.groupby(['machine', 'point'])['rms_mms'].apply(lambda x: x.shift().expanding().mean())
result['mean_mms_prev'] = result['mean_mms_prev']/result['rms_mms']
result = result.fillna(0)
result['element_type'] = result['point'][0:1] 
result.loc[:, 'element_type'] = result.point.map(lambda x: x[0])
result.index = result.time
result = result.dropna().sort_index()
result['Id_Estado_Activo_fixed'] = result['Id_Estado_Activo_fixed'].astype('int')
result['prev_diag'] = result['prev_diag'].astype('int')

# We set the Good status as 0 label and Warning and Alarm as label 1.
result['prev_diag'] = result['prev_diag'].replace({1:0, 2:1, 3:1})
result['Id_Estado_Activo_fixed'] = result['Id_Estado_Activo_fixed'].replace({1:0, 2:1, 3:1})

In [56]:
result.prev_diag.value_counts()

0    7011
1    1529
Name: prev_diag, dtype: int64

In [57]:
result_pred = result[['max_mms', 'rms_mms', 'max_acc', 'med_acc',
             'rms_acc', 'kur_acc', 'cf_acc', 'num_peaks', 'max_dem', 'rms_dem','pct_change_acc',
             'a_range', 'b_range', 'c_range', 'd_range', 'e_range', 'f_range', 'g_range', 'h_range',
             'i_range', 'j_range', 'mean_mms_prev', 'element_type', 'prev_diag']]
x_pred = ct.transform(result_pred)

In [59]:
model_filename = 'Model/gcbmodel.sav'
model = pickle.load(open(model_filename, 'rb'))

In [60]:
y_pred = model.predict(x_pred)

In [68]:
import numpy as np
np.unique(y_pred)

array([0, 1])

In [69]:
result['prediction'] = y_pred

In [75]:
result[result['prediction']==1].groupby('machine').count()

,point,time,max_mms,rms_mms,max_acc,med_acc,rms_acc,kur_acc,cf_acc,num_peaks,...,j_range,max_dem,rms_dem,Id_Estado_Activo,Id_Estado_Activo_fixed,prev_diag,pct_change_acc,mean_mms_prev,element_type,prediction
machine,,,,,,,,,,,,,,,,,,,,,
S1-M1,17,17,17,17,17,17,17,17,17,17,...,17,17,17,17,17,17,17,17,17,17
S1-M11,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
S1-M12,3,3,3,3,3,3,3,3,3,3,...,3,3,3,3,3,3,3,3,3,3
S1-M14,5,5,5,5,5,5,5,5,5,5,...,5,5,5,5,5,5,5,5,5,5
S1-M15,4,4,4,4,4,4,4,4,4,4,...,4,4,4,4,4,4,4,4,4,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
S6-77-M2,10,10,10,10,10,10,10,10,10,10,...,10,10,10,10,10,10,10,10,10,10
S6-78-M1,11,11,11,11,11,11,11,11,11,11,...,11,11,11,11,11,11,11,11,11,11
S6-78-M2,17,17,17,17,17,17,17,17,17,17,...,17,17,17,17,17,17,17,17,17,17
